In [6]:
import math
import mysql.connector as mysql
import numpy as np
import os
import pandas as pd
import re
import urllib.request

from os.path import splitext
from pathlib import Path
from PIL import ImageFile

In [8]:
# GENERAL CONFIGS
ImageFile.LOAD_TRUNCATED_IMAGES = True

# == Database config ==
conn = mysql.connect(
  host="localhost",
  user="root",
  password="root"
)

cursor = conn.cursor()

# == Files config ==
FILE_FOLDER = r"D:\legal_data\myntra.csv"
fdata=pd.read_csv(Path(FILE_FOLDER), error_bad_lines=False)
IMG_FOLDER_DEST = r"D:\images\_myntra"

DB_NAME = "blp_dataset"
TABLE_NAME = "products_tmp_myntra"

b'Skipping line 20: expected 25 fields, saw 26\nSkipping line 106: expected 25 fields, saw 26\nSkipping line 116: expected 25 fields, saw 26\nSkipping line 119: expected 25 fields, saw 26\nSkipping line 132: expected 25 fields, saw 26\nSkipping line 139: expected 25 fields, saw 26\nSkipping line 239: expected 25 fields, saw 26\nSkipping line 284: expected 25 fields, saw 26\nSkipping line 294: expected 25 fields, saw 26\nSkipping line 332: expected 25 fields, saw 26\nSkipping line 389: expected 25 fields, saw 26\nSkipping line 434: expected 25 fields, saw 26\nSkipping line 438: expected 25 fields, saw 26\nSkipping line 439: expected 25 fields, saw 26\nSkipping line 453: expected 25 fields, saw 26\nSkipping line 491: expected 25 fields, saw 26\nSkipping line 507: expected 25 fields, saw 26\nSkipping line 524: expected 25 fields, saw 26\nSkipping line 533: expected 25 fields, saw 26\nSkipping line 563: expected 25 fields, saw 26\nSkipping line 627: expected 25 fields, saw 26\nSkipping lin

In [9]:
# Creating environment
cursor.execute("CREATE DATABASE IF NOT EXISTS {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
cursor.execute("USE {}".format(DB_NAME))

cursor.execute("DROP TABLE IF EXISTS products_tmp_myntra")

In [10]:
table_schema = (
    "CREATE TABLE IF NOT EXISTS products_tmp_myntra ("
    "  id INT(11) AUTO_INCREMENT PRIMARY KEY NOT NULL,"
    "  name TEXT NOT NULL,"
    "  description TEXT,"
    "  market_name VARCHAR(255),"
    "  seller_name TEXT,"
    "  illegal ENUM('t', 'f') NOT NULL DEFAULT 'f'"
    ") ENGINE=InnoDB")

cursor.execute(table_schema)

In [11]:
fdata.columns

Index(['uniq_id', 'crawl_timestamp', 'product_id', 'link', 'size',
       'variant_sku', 'brand', 'care_instructions', 'dominant_material',
       'title', 'actual_color', 'dominant_color', 'product_type', 'images',
       'body', 'product_details', 'size_fit', 'complete_the_look', 'type',
       'variant_price', 'variant_compare_at_price', 'ideal_for', 'is_in_stock',
       'inventory', 'specifications'],
      dtype='object')

In [12]:
# Deleting unwanted columns
del_list = ['uniq_id', 'crawl_timestamp', 'product_id', 'link', 'size', 'variant_sku', 'brand',
            'care_instructions', 'dominant_material', 'actual_color', 'dominant_color', 'product_type', 'body', 
            'size_fit', 'complete_the_look', 'type', 'variant_price', 'variant_compare_at_price', 'ideal_for', 'is_in_stock', 
            'inventory', 'specifications']

fdata = fdata.drop(del_list, axis = 1)

fdata.columns

Index(['title', 'images', 'product_details'], dtype='object')

In [13]:
def clear_text(text):
    clear_content = re.sub(r'[\t\r\n]', ' ', text)
    clear_content = re.sub(' +',' ', clear_content).lstrip()

    return clear_content

In [14]:
def insert_product(row):   
    encoded_string = row['product_details'].encode('utf-8', "ignore")
    description = encoded_string.decode()
    
    description = description.replace("Make sure this fits by entering your model number. | ", "")
    description = re.sub('[👍🌍📱✅【】👐🔬⚗️⚙️🧜😎🔴🔵🙌🌿🎁🦄💃👜⭐💦]', '', description)
    description = clear_text(description.replace(' | ', ''))
    
    query = "INSERT INTO products_tmp_myntra (name, description, market_name, illegal) VALUES (%s, %s, %s, %s)"
    values = (row['title'], description, "Myntra", "f")

    cursor.execute(query, values)
    conn.commit()

    return cursor.lastrowid

In [15]:
for index, row in fdata.iterrows():
    if row['title'] == "NaN" or type(row['title']) is not str: row['title'] = ""
    if row['product_details'] == "NaN" or type(row['product_details']) is not str: row['product_details'] = ""

    if row['title'] is not "" and row['product_details'] is not "":
        product_id = insert_product(row)

        image = row['images'].split(sep="|")[0]

        folder_dest = Path(f"{IMG_FOLDER_DEST}\{product_id}.jpeg")

        try:
            file = urllib.request.urlretrieve(image, folder_dest)
        except:
            print(f"Product {product_id}: Error retrieving image ({image})")

Product 6278: Error retrieving image (http://assets.myntassets.com/v1/assets/images/1803398/2017/4/4/11491287105986-AKKRITI-BY-PANTALOONS-Women-Blue--Red-Semi-Sheer-Printed-Top-3041491287105700-1.jpg )
Product 12653: Error retrieving image (http://assets.myntassets.com/v1/assets/images/7139141/2019/2/20/d192c547-4e29-490b-ab5a-16f88b9cf4191550657129938-Anouk-Women-Grey--Maroon-Printed-Kurta-with-Churidar--Dupatt-1.jpg )
